<a href="https://colab.research.google.com/github/hhk54250/Is20f/blob/master/first%20try%20on%20HJM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
from scipy import linalg as la



class PrincipalComponentFactors(object):
    def __init__(self, dataframe):
        self.tenors     = [float(x) for x in dataframe.columns]
        hist_rates      = np.matrix(dataframe)
        self.diff_rates = np.diff(hist_rates, axis=0)
        
        print("Data shape : " + str(self.diff_rates.shape))
        #Calculate covariance matrix

        self.sigma = np.cov(self.diff_rates.transpose())
        print("Variance Covariance shape : " + str(self.sigma.shape))

        # Calculate eigenvalues and eigenvectors
        evals, evecs = la.eig(self.sigma)
        self.evecs   = np.matrix(evecs)

        # check the shapes
        print("eigvalue  shape : " + str(evals.shape))
        print("eigvector shape : " + str(evecs.shape))

        # Determine the largest three principal components
        factors = 3
        index_evecs     = list(reversed(np.real(evals).argsort()))[0:factors]
        self.princ_evals= np.array([evals[i] for i in index_evecs])
        princ_comp      = np.hstack([self.evecs[:,i] for i in index_evecs])
        self.princ_comp = np.real(princ_comp)
        # check the shapes
        print("PC shape : " + str(self.princ_comp.shape))
        
    def VolatilityFactors(self):        
        sqrt_evals         = np.matrix(self.princ_evals ** .5)
        tmp_m              = np.vstack([sqrt_evals for i in range(self.princ_comp.shape[0])]) 
        self.daily_factors = np.multiply(np.real(tmp_m), self.princ_comp)
        self.annual_factors= self.daily_factors * np.sqrt(256)
        
        v1                 = np.array(self.daily_factors[:,0].flatten())[0]
        v2                 = np.array(self.daily_factors[:,1].flatten())[0]
        v3                 = np.array(self.daily_factors[:,2].flatten())[0]
        
        # Take the square to calculate the daily volatility
        self.daily_vols = np.sqrt(v1**2 + v2**2 + v3**2)
        # Multiply by sqrt of 252 to calculate the annual volatility
        self.ann_vols   = (self.daily_vols) *np.sqrt(256)
        
        
        print('daily vols shape: ' + str(self.daily_vols.shape))   
       
    def PlotPC(self):
        # Print Principal Components
        plt.figure(figsize=(10,4))
        plt.plot(self.tenors, self.princ_comp, marker='.'), plt.title('Principal components'), plt.xlabel(r'Tenor $\tau$');
        plt.grid(True)
        plt.show()
        
    def PlotVF(self):
        # Print Daily Volatility Factor. We prefer to print the daily factors as this preserves the factor's sign
        plt.figure(figsize=(10,4))
        plt.plot(self.tenors, self.daily_factors, marker='.'), \
        plt.title('Daily Volatility Factors'), \
        plt.xlabel(r'Tenor $\tau$'), plt.ylabel(r'Daily Volatility $\sigma$');
        plt.grid(True)
        plt.show()